In [1]:
import torch

from gfos.model.gnn import LayoutModel
from gfos.data.dataset import LayoutDataset, Normalizer
from gfos.data.utils import load_layout

In [2]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"
NORMALIZER_PATH = "../../data/normalizer.json"

SOURCE = "xla"
SEARCH = "default"

In [3]:
normalizer = Normalizer.from_json(NORMALIZER_PATH, SOURCE, SEARCH)

layout_data = load_layout(
    LAYOUT_DIR,
    model_type=SOURCE,
    compile_type=SEARCH,
)

test_dataset = LayoutDataset(
    layout_data["valid"],
    normalizer=normalizer,
)

Loading data: 100%|██████████| 7/7 [00:10<00:00,  1.50s/it, unet_3d.4x4.bf16]                    


In [7]:
model = LayoutModel(
    node_feat_dim=112,
    node_config_dim=14,
    config_neighbor_layer="GATConv",
    dropout=0.2,
    num_config_layers=3,
    config_neighbor_dim=32,
    node_dropout_between_layers=0.1,
    config_neighbor_dropout_between_layers=0.1,
    config_dropout_between_layers=0.1,
)
state_dict = torch.load(
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_130700\runs\2023-10-25_13-07-00\wandb\run-20231025_130818-j33ecz9e\files\239_0.4217.pth"
)
model.load_state_dict(state_dict["state_dict"])

Disable config edge weight


<All keys matched successfully>

In [9]:
model = model.train().cuda()

In [ ]:
model = model.eval().cuda()

In [10]:
from gfos.metrics import LayoutMetrics

In [11]:
import tqdm
device = "cuda"
batch_size = 100

results = {}
metric = LayoutMetrics()

with torch.no_grad():
    for record in tqdm.tqdm(test_dataset):
        model_id = record["model_id"]
        node_feat = record["node_feat"]
        node_opcode = record["node_opcode"]
        edge_index = record["edge_index"]
        node_config_feat = record["node_config_feat"]
        node_config_ids = record["node_config_ids"]
        config_runtime = record["config_runtime"]
        config_edge_index = record["config_edge_index"]
        config_edge_weight = record["config_edge_weight"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            config_edge_index,
            config_edge_weight,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
            config_edge_index.to(device),
            config_edge_weight.to(device),
        )

        c = len(config_runtime)
        outs = []

        for i in range(0, c, batch_size):
            end_i = min(i + batch_size, c)
            blend = []
            for k in range(50):
                out: torch.Tensor = model(
                    node_feat,
                    node_opcode,
                    edge_index,
                    node_config_feat[i:end_i],
                    node_config_ids,
                    config_edge_index,
                    config_edge_weight,
                )
                blend.append(out)
            out = torch.mean(torch.stack(blend), dim=0)
            outs.append(out.detach().cpu())
        
        outs = torch.concat(outs)
        metric.add(model_id, outs.numpy(), config_runtime.numpy())
        results[model_id] = outs

100%|██████████| 7/7 [15:57<00:00, 136.74s/it]


In [12]:
metric.compute_scores()

{'raw_kendall': 0.41901337566528024,
 'index_kendall': 0.41900964305206073,
 'top100_error': 0.9914285714285717,
 'top500_error': 0.8505714285714285,
 'kendall_bert_pretraining.4x4.fp16': 0.5195674582021488,
 'kendall_inception_v3_batch_128_train': 0.6456680738026904,
 'kendall_mlperf_bert_batch_24_2x2': 0.31575607476880835,
 'kendall_resnet50.4x4.fp16': 0.516374797453982,
 'kendall_resnet_v1_50_official_batch_128_bf16': 0.4449971531755013,
 'kendall_tf2_bert_pretrain_dynamic_batch_size': 0.41828384269835744,
 'kendall_unet_3d.4x4.bf16': 0.07242010126293642}

In [ ]:
import pickle


pickle.dump(results, open("results.pkl", "wb"))